<a href="https://colab.research.google.com/github/psakore/IMCFeedArbitration/blob/main/imc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scapy
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

In [ ]:
import os
if not os.path.exists('feed_arbitrage.pcap'):
  !wget https://github.com/psakore/IMCFeedArbitration/raw/main/feed_arbitrage.pcap


In [144]:
import sys
from scapy.all import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import interact, widgets, Output
from IPython.display import display


MAX_TIME = 9999999999999999

In [145]:
def process_packet(packet, data_dict):
    # Extract relevant information from the packet (symbol, seqno, timestamp)
    payload = packet.load.decode("utf-8")
    symbol = re.search(r"Symbol: (\w+)", payload).group(1)
    seqno = int(re.search(r"Seqno: (\d+)", payload).group(1))
    timestamp = int(packet.time * 1000000) # converting to int
    source_ip = packet[IP].src  # Get the source IP address

    # Add the packet information to the data dictionary
    key = "%s_%d" % (symbol, seqno)
    if key not in data_dict:
      data_dict[key] = {}
    data_dict[key][source_ip] = timestamp

In [146]:
def calculate_metrics(data_dict, num_best):
    publishers_tally = {}
    for symbol, symbol_data in data_dict.items():
        data_dict[symbol]['fastest_pub'] = min(symbol_data['publishers'], key=symbol_data['publishers'].get)


In [147]:
def main(pcap_file, num_best):
    # Initialize a data dictionary to store packet information
    data_dict = {}

    # Iterate through packets, process them, and calculate metrics
    for packet in PcapReader(pcap_file):
        if packet.haslayer(UDP):
            process_packet(packet, data_dict)

    df = pd.DataFrame(data_dict).T
    df = df.reindex(sorted(df.columns), axis=1)
    df.index.name = 'Symbol_Seqno'
    df.fillna(MAX_TIME, inplace=True)
    df = df.astype(int)

    df['Fastest_Publisher'] = df.idxmin(axis=1)


    top_values = df['Fastest_Publisher'].value_counts().head(num_best).index.tolist()

    # Filter the DataFrame to include only columns in top_values
    filtered_df = df[top_values]
    min_time_df = filtered_df.min(axis=1)

    #print(min_time_df)

main('feed_arbitrage.pcap', 2)